<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:
- Replicate the lesson code.

In [0]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

In [0]:
re = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv')

In [141]:
re['LotFrontage'].value_counts(dropna=False).head()

NaN     259
60.0    143
70.0     70
80.0     69
50.0     57
Name: LotFrontage, dtype: int64

In [142]:
type(np.NaN)

float

In [0]:
# Calculate Total SF
re['TotalSF'] = re['TotalBsmtSF'] + re['1stFlrSF'] + re['2ndFlrSF']

In [153]:
# Calculate Price / SQFT
re['$/SF'] = re['SalePrice'] / re['TotalSF']
re.iloc[0:5,-2:]

,TotalSF,$/SF
0,2566,81.254871
1,2524,71.909667
2,2706,82.594235
3,2473,56.611403
4,3343,74.783129


In [0]:
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
!unzip LoanStats_2018Q4.csv.zip

In [0]:
# Sets header, and skips bottom two rows
loans = pd.read_csv('LoanStats_2018Q4.csv', header=1, skipfooter=2, engine='python')

In [156]:
# Counts number of isnull values for each column
loans.isnull().sum().sort_values()

inq_fi                                             0
delinq_amnt                                        0
chargeoff_within_12_mths                           0
acc_open_past_24mths                               0
inq_last_12m                                       0
total_cu_tl                                        0
total_rev_hi_lim                                   0
max_bal_bc                                         0
open_rv_24m                                        0
open_rv_12m                                        0
total_bal_il                                       0
open_il_24m                                        0
hardship_flag                                      0
open_act_il                                        0
open_acc_6m                                        0
tot_cur_bal                                        0
tot_coll_amt                                       0
acc_now_delinq                                     0
application_type                              

In [0]:
# DROPS COLUMNS WITH NO DATA
loans = loans.drop(['url', 'member_id', 'desc', 'id'], axis=1)

In [158]:
# SHOWS A SAMPLE OF INTEREST RATE
loans['int_rate'][3:7]

3     12.98%
4     12.98%
5      7.56%
6     17.97%
Name: int_rate, dtype: object

In [0]:
# FUNCTION TO CONVERT INTEREST RATE DATA TO FLOAT
def ir_to_float(cell):
  return float(cell.strip().strip('%'))

In [0]:
# CREATES NEW COLUMN WITH FLOAT VALUE
loans['ir_to_float'] =  loans['int_rate'].apply(ir_to_float)

In [0]:
# CONFIRMS NEW DATA IS TYPE FLOAT
loans.dtypes[-5:]

settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
ir_to_float              float64
dtype: object

In [0]:
# SUMMARY OF EMPLOYMENT TITLES
loans['emp_title'].value_counts(dropna=False)[:10]

NaN                 20947
Teacher              2090
Manager              1773
Registered Nurse      952
Driver                924
RN                    726
Supervisor            697
Sales                 580
Project Manager       526
General Manager       523
Name: emp_title, dtype: int64

In [0]:
# FUNCTION TO CLEAN EMPLOYMENT TITLES
def clean_title(title_input):
  if isinstance(title_input, str):
    return title_input.title().strip()
  else:
    return "Unknown"

In [160]:
# TEST CASES FOR clean_title FUNCTION
errors = ['owner', 'Supervisor', ' Project Manager', np.NaN]
[clean_title(i) for i in errors]

['Owner', 'Supervisor', 'Project Manager', 'Unknown']

In [0]:
# UPDATES DATAFRAME WITH CLEANED DATA
loans['emp_title'] = loans['emp_title'].apply(clean_title)

In [163]:
# NaN's CHANGED TO Unknown
loans['emp_title'].value_counts(dropna=False, ascending=False)[:10]

Unknown             20947
Teacher              2557
Manager              2395
Registered Nurse     1418
Driver               1258
Supervisor           1160
Truck Driver          920
Rn                    834
Office Manager        805
Sales                 803
Name: emp_title, dtype: int64

In [164]:
print(len(loans['emp_title'].value_counts()))
print(loans['emp_title'].isnull().sum())

34902
0


In [165]:
[col for col in loans if col.endswith('_d')]

['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

In [166]:
# CONVERTS "ISSUE_D" TO DATETIME FORMAT
loans['issue_d'] = pd.to_datetime(loans['issue_d'], infer_datetime_format=True)
loans.dtypes[:15]

loan_amnt                       int64
funded_amnt                     int64
funded_amnt_inv               float64
term                           object
int_rate                       object
installment                   float64
grade                          object
sub_grade                      object
emp_title                      object
emp_length                     object
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                datetime64[ns]
loan_status                    object
dtype: object

In [0]:
# ADDS COLUMNS FOR ISSUE DATE MONTH AND YEAR
loans['issue_year'] = loans['issue_d'].dt.year
loans['issue_month'] = loans['issue_d'].dt.month

In [169]:
# CALCULATES DAYS FROM THE EARLIEST CREDIT LINE
loans['earliest_cr_line'] = pd.to_datetime(loans['earliest_cr_line'], infer_datetime_format=True)
loans['days_from_earliest_cl_issue'] = (loans['issue_d'] - loans['earliest_cr_line']).dt.days
loans['days_from_earliest_cl_issue'].head()

0    14428
1     5570
2     4383
3     1979
4    12144
Name: days_from_earliest_cl_issue, dtype: int64

In [0]:
# CALCULATES AVERAGE LENGTH OF CREDIT HISTORY IN YEARS
loans['days_from_earliest_cl_issue'].mean() / 365

16.054497232580577

In [0]:
# CALCULATES LONGEST CREDIT HISTORY IN DATASET
loans['days_from_earliest_cl_issue'].max() / 365

68.96164383561644

**Convert the `term` column from string to integer.**

In [0]:
# NO NAN VALUES, ONLY 36 OR 60 MONTHS
loans['term'].value_counts(dropna=False, ascending=False)

 36 months    88179
 60 months    40233
Name: term, dtype: int64

In [0]:
# FUCTION TO CONVERT 'CLEAN' COLUMN TO INT
def clean_term(cell):
  return int(cell.strip(' months'))

In [170]:
# APPLIES FUNCTION
loans['term_i'] = loans['term'].apply(clean_term)
loans['term_i'].dtype

dtype('int64')

**Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.**

In [173]:
# VALUES OF loan_status COLUMN
loans['loan_status'].value_counts(dropna=False, ascending=False)

Current               100571
Fully Paid             19545
Charged Off             4433
Late (31-120 days)      2662
In Grace Period          730
Late (16-30 days)        467
Default                    4
Name: loan_status, dtype: int64

In [0]:
# FUNCTION TO RETURN 1 IF CURRENT OR FULLY PAID, ELSE 0
def clean_loan(cell):
  if cell == 'Current' or cell == 'Fully Paid':
    return 1
  else:
    return 0

In [179]:
# TEST CASES FOR clean_loan FUNCTION
loan_examples = ['Current', 'Fully Paid', 'Charged Off', 'Late (31-120 days)', 
                 'In Grace Period', 'Late (16-30 days)', 'Default']
[clean_loan(i) for i in loan_examples]

[1, 1, 0, 0, 0, 0, 0]

In [0]:
 # APPLIES FUNCTION TO LOANS DATAFRAME
loans['loan_status_is_great'] = loans['loan_status'].apply(clean_loan)
# loans['loan_status_is_great'].head(30)

**Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.**

In [199]:
# ADDED COLUMNS, RETURNS FLOAT??
loans['last_pymnt_d'] = pd.to_datetime(loans['last_pymnt_d'], infer_datetime_format=True, errors='coerce')
loans['last_pymnt_d_month'] = loans['last_pymnt_d'].dt.month
loans['last_pymnt_d_year'] = loans['last_pymnt_d'].dt.year
loans.iloc[0:5,-2:]

,last_pymnt_d_month,last_pymnt_d_year
0,11.0,2019.0
1,4.0,2019.0
2,10.0,2019.0
3,10.0,2019.0
4,4.0,2019.0


In [200]:
loans['last_pymnt_d'].value_counts(dropna=False, ascending=False)

2019-11-01    94238
2019-10-01    10607
2019-07-01     3156
2019-09-01     2871
2019-08-01     2758
2019-05-01     2547
2019-04-01     2356
2019-06-01     2285
2019-03-01     1984
2019-02-01     1915
2019-01-01     1561
2018-12-01     1054
2018-11-01      679
2018-10-01      234
NaT             159
2019-12-01        8
Name: last_pymnt_d, dtype: int64

# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

In [0]:
test_cases = ['15.2%', '  50.1', np.NaN]

In [0]:
def clean_util(cell):
  if isinstance(cell, str):
    return float(cell.strip().strip('%'))/100
  else:
    return np.NaN

In [0]:
[clean_util(i) for i in test_cases]

[0.152, 0.501, nan]

In [0]:
loans['revol_float'] = loans['revol_util'].apply(clean_util)
loans.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,ir_to_float,issue_year,issue_month,days_from_earliest_cl_issue,revol_float
0,20000,20000,20000.0,36 months,14.47%,688.13,C,C2,Bus Driver,4 years,OWN,52000.0,Source Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,1979-06-01,2,15.0,NaN,6,0,15048,73%,22,w,14777.99,14777.99,7481.000000,7481.00,5222.01,2258.99,0.0,0.0,...,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,14428,0.730
1,25000,25000,25000.0,60 months,16.14%,609.82,C,C4,Production Manager,5 years,MORTGAGE,45000.0,Not Verified,2018-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,703xx,LA,37.09,0,2003-09-01,0,NaN,NaN,7,0,8901,36.8%,21,w,0.00,0.00,26653.167580,26653.17,25000.00,1653.17,0.0,0.0,...,18600,49483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,16.14,2018,12,5570,0.368
2,26500,26500,26500.0,60 months,11.31%,580.28,B,B3,Compliance Director,10+ years,MORTGAGE,134000.0,Source Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,747xx,OK,18.91,0,2006-12-01,2,NaN,NaN,17,0,43640,65.4%,37,w,23051.06,23051.06,5777.820000,5777.82,3448.94,2328.88,0.0,0.0,...,49300,88875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,11.31,2018,12,4383,0.654
3,10400,10400,10400.0,36 months,12.98%,350.32,B,B5,Program Support Assistant,10+ years,MORTGAGE,56099.0,Source Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,800xx,CO,20.92,0,2013-07-01,2,32.0,67.0,8,1,1669,9.8%,10,w,7902.57,7902.57,3491.950000,3491.95,2497.43,994.52,0.0,0.0,...,17000,43709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98,2018,12,1979,0.098
4,35000,35000,35000.0,60 months,12.98%,796.00,B,B5,Sr. Vice President -Risk Manager,7 years,MORTGAGE,211646.0,Source Verified,2018-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,797xx,TX,15.09,0,1985-09-01,0,77.0,NaN,13,0,55944,74.9%,39,w,0.00,0.00,36266.741552,36266.74,35000.00,1266.74,0.0,0.0,...,59700,51904,88364.0,Sep-1985,0.0,6.0,19.0,53.4,2.0,29.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98,2018,12,12144,0.749
